# <center> Segmenting and Clustering Neighborhoods in Vancouver

In this project I explored, segmented, and clustered the neighborhoods in the city of Toronto.
For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. I scraped the Wikipedia page and wrangled the data, cleaned it, and then read it into a pandas dataframe so that it is in a structured format.
Once the data is in a structured format, I explored and clustered the neighborhoods in the city of Toronto.

## Part 1: Creating a notebook by scraping the wikipedia page containing Toronto neighborhood info

In [77]:
#installing Folium
import sys
!{sys.executable} -m pip install folium

In [78]:
#installing Folium
import sys
!{sys.executable} -m pip install geopy

In [79]:
#importing required libraries for this project

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [80]:
import requests
import pandas as pd 

data from wikipedia is scareped and converted to a pandas dataframe

In [81]:
df0=pd.read_csv('Vancouver_PostalCode.csv')

In [82]:
df1= pd.read_csv('addresses.csv')


In [83]:
df0.head()

,PostalCode,Borough,Neighborhood
0,V5K,Vancouver,"North Hastings, Sunrise"
1,V5L,Vancouver,"North grandview, Woodland"
2,V5M,Vancouver,"South Hastings, Sunrise, North Renfrew- Collin..."
3,V5N,Vancouver,"South Grandview, Woodland, Ne Kensington, Ceda..."
4,V5P,Vancouver,"SE Kensington, Cedar Cottage, Victoria drive, ..."


In [84]:
df1=df1[['X', 'Y', 'PostalCode']]

In [85]:
df1.head()

,X,Y,PostalCode
0,-123.142289,49.303832,V5K
1,-123.142289,49.303832,V5L
2,-123.034651,49.249830,V5M
3,-123.067986,49.249830,V5N
4,-123.067984,49.216496,V5P


In the next few columns I renamed some columns and reindex the dataframe

In [86]:
Vancouver_Geo=df1.rename({'X': 'Latitude', 'Y': 'Longitude'}, axis=1)

In [87]:
Vancouver_Ne=df0.rename({'Borough ': 'Borough'}, axis=1)

In [88]:
Vancouver_Ne.groupby('PostalCode')

In [89]:
Vancouver_Geo.columns

Index(['Latitude', 'Longitude', 'PostalCode'], dtype='object')

In [90]:
Vancouver_Geo=Vancouver_Geo.rename({'Latitude':'Longitude', 'Longitude':'Latitude'}, axis=1)

In [91]:
Vancouver_Geo.head()

,Longitude,Latitude,PostalCode
0,-123.142289,49.303832,V5K
1,-123.142289,49.303832,V5L
2,-123.034651,49.249830,V5M
3,-123.067986,49.249830,V5N
4,-123.067984,49.216496,V5P


In [92]:
Vancouver_Ne.head()

,PostalCode,Borough,Neighborhood
0,V5K,Vancouver,"North Hastings, Sunrise"
1,V5L,Vancouver,"North grandview, Woodland"
2,V5M,Vancouver,"South Hastings, Sunrise, North Renfrew- Collin..."
3,V5N,Vancouver,"South Grandview, Woodland, Ne Kensington, Ceda..."
4,V5P,Vancouver,"SE Kensington, Cedar Cottage, Victoria drive, ..."


In [93]:
Vancouver_Geo.groupby('PostalCode')

In [94]:
Vancouver_Ne.shape

(31, 3)

In [95]:
Vancouver_Geo.shape

(30, 3)

In [96]:
Van_data=Vancouver_Ne.merge(Vancouver_Geo, on='PostalCode', how='inner')


In [97]:
Van_data

,PostalCode,Borough,Neighborhood,Longitude,Latitude
0,V5K,Vancouver,"North Hastings, Sunrise",-123.142289,49.303832
1,V5L,Vancouver,"North grandview, Woodland",-123.142289,49.303832
2,V5M,Vancouver,"South Hastings, Sunrise, North Renfrew- Collin...",-123.034651,49.249830
3,V5N,Vancouver,"South Grandview, Woodland, Ne Kensington, Ceda...",-123.067986,49.249830
4,V5P,Vancouver,"SE Kensington, Cedar Cottage, Victoria drive, ...",-123.067984,49.216496
5,V5R,Vancouver,"South Renfrew, Collingwood",-123.139155,49.300742
6,V5S,Vancouver,Killarney,-123.034650,49.216497
7,V5T,Vancouver,East Mount Pleasant,-123.154222,49.226656
8,V5V,Vancouver,"West Kensington, Cedar Cottage, North East Ri...",-123.075078,49.249125
9,V5W,Vancouver,"South East Riley, Park Little Mountain, SW Ken...",-123.084651,49.216496


In [98]:
Van_data.columns

Index(['PostalCode', 'Borough', 'Neighborhood', 'Longitude', 'Latitude'], dtype='object')

In [99]:
Van_data.shape

(30, 5)

In [100]:
len (Van_data['Borough'].unique())

2

## Part 2: Finding the geographical locations (lat and long) of each neighborhood in Toronto

merged the two dataframes (The one including the neighborhood data and the one including the geographical data) 

In [101]:
address = 'Vancouver'

geolocator = Nominatim(user_agent="Van_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Vancouver are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Vancouver are 49.2608724, -123.1139529.


In [102]:
# create map of New York using latitude and longitude values
map_Vancouver = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Van_data['Latitude'], Van_data['Longitude'], Van_data['Borough'], Van_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Vancouver)  
    
map_Vancouver

**Created the map of Toronto using the geographcal data**

## Part 3 Use FourSquare to explore and cluster venues in Toronto 

In this section:
1. I used FourSquare to get 100 venues in Toronto 
2. Explored and cluster the neighborhoods in Toronto
3. Generated maps to visualize neighborhoods and how they cluster together

In [103]:
CLIENT_ID = 'VP35KICHELLPBIIZ4FGYVHJHBC2F4WWFXVNBMLXR4BUXJOTB' # your Foursquare ID
CLIENT_SECRET = 'DKQ55RQV5WLNTFAUMK04GTFF4UM34CHBIO5FA2ZIXH3HA1V2' # your Foursquare Secret
VERSION = '20200531'

In [104]:
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VP35KICHELLPBIIZ4FGYVHJHBC2F4WWFXVNBMLXR4BUXJOTB
CLIENT_SECRET:DKQ55RQV5WLNTFAUMK04GTFF4UM34CHBIO5FA2ZIXH3HA1V2


#### Let's explore the first neighborhood in our dataframe.

In [105]:
Van_data.loc[0, 'Neighborhood']

'North Hastings, Sunrise'

In [106]:
neighborhood_latitude = Van_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Van_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Van_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of North Hastings, Sunrise are 49.3038316, -123.14228870000001.


#### Now, let's get the top 100 venues that are in Toronto within a radius of 500 m.

In [107]:
search_query= 'Top 100 Venues'
LIMIT=100
radius=500

In [108]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [109]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ee7d671dff54e5fd8767b98'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Stanley Park',
  'headerFullLocation': 'Stanley Park, Vancouver',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 6,
  'suggestedBounds': {'ne': {'lat': 49.308331604500005,
    'lng': -123.13540024313264},
   'sw': {'lat': 49.2993315955, 'lng': -123.14917715686738}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bf9d57e8f32ef3b12c504aa',
       'name': 'Stanley Park',
       'location': {'address': 'Stanley Park Dr',
        'lat': 49.302488447038236,
        'lng': -123.14171791076662,
        'labeledLatLngs': [{'label': 'display',
          'lat': 

**As all the information is in the 'items' key, I used the 'get_category_type' **function.**

In [110]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Then clean the json file and structure it into a *pandas* dataframe.

In [111]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Stanley Park,Park,49.302488,-123.141718
1,Wren Trail,Trail,49.302527,-123.142399
2,Propesct Point Picnic Site,Park,49.303014,-123.146194
3,Beaver Lake,Lake,49.305053,-123.138252
4,Squirrel trail,Trail,49.303553,-123.147097
5,Beaver Lake Trail,Trail,49.303440,-123.137298


In [112]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

6 venues were returned by Foursquare.


The function below will repeat the same process to all the neighborhoods in Toronto

In [113]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [114]:
Vancouver_venues = getNearbyVenues(names=Van_data['Neighborhood'],
                                   latitudes=Van_data['Latitude'],
                                   longitudes=Van_data['Longitude']
                                  )



North Hastings, Sunrise
North grandview, Woodland
South Hastings, Sunrise, North Renfrew- Collingwod
South Grandview, Woodland, Ne Kensington, Cedaar Cottage
SE Kensington, Cedar Cottage, Victoria drive, Fraserview 
South Renfrew, Collingwood
Killarney
East Mount Pleasant
 West Kensington, Cedar Cottage, North East Riley Park, Little Mountain 
South East Riley, Park Little Mountain, SW Kensington, Cedar Cottage, North East Oakridge, North Sunset
SE Oakridge, East Marpole, South Sunset
West Mount Pleasant, West Riley Park, Little Mountain 
East Fairview, South Cambie 
Strathcona 
Waterfron, Coal Harbour, Canada Place
South East West End, Davie Village
North West West End, Stanely Park
West Fairview, Granvile Iceland, Nort East Shaughnessy 
North West Shaughnessy, East Kitsilano, Quilchena 
Central Kitsilano, Greektown 
North West Arbutus Ridge, North East Dunbar, Southlands
South Shaughnessy, North West Oakridge, North East Kerrisdale, South East Arbutus Ridge
West Kerisdale, South Dunb

In [115]:
print(Vancouver_venues.shape)
Vancouver_venues.head()

(696, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"North Hastings, Sunrise",49.303832,-123.142289,Stanley Park,49.302488,-123.141718,Park
1,"North Hastings, Sunrise",49.303832,-123.142289,Wren Trail,49.302527,-123.142399,Trail
2,"North Hastings, Sunrise",49.303832,-123.142289,Propesct Point Picnic Site,49.303014,-123.146194,Park
3,"North Hastings, Sunrise",49.303832,-123.142289,Beaver Lake,49.305053,-123.138252,Lake
4,"North Hastings, Sunrise",49.303832,-123.142289,Squirrel trail,49.303553,-123.147097,Trail


In [116]:
Vancouver_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"West Kensington, Cedar Cottage, North East Riley Park, Little Mountain",24,24,24,24,24,24
Bental Center,18,18,18,18,18,18
"Central Kitsilano, Greektown",30,30,30,30,30,30
"East Fairview, South Cambie",30,30,30,30,30,30
East Mount Pleasant,9,9,9,9,9,9
Killarney,9,9,9,9,9,9
"North Hastings, Sunrise",6,6,6,6,6,6
"North West Arbutus Ridge, North East Dunbar, Southlands",16,16,16,16,16,16
"North West Dunbar, Southlands, Chaldecutt, South University Endowment Lands",79,79,79,79,79,79


In [117]:
print('There are {} uniques categories.'.format(len(Vancouver_venues['Venue Category'].unique())))

There are 150 uniques categories.


In [118]:
 #one hot encoding
Vancouver_onehot = pd.get_dummies(Vancouver_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Vancouver_onehot['Neighborhood'] = Vancouver_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Vancouver_onehot.columns[-1]] + list(Vancouver_onehot.columns[:-1])
Vancouver_onehot = Vancouver_onehot[fixed_columns]

Vancouver_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Arts & Crafts Store,Asian Restaurant,Bagel Shop,Bakery,Bank,Bar,Beach,Big Box Store,Bistro,Boat Rental,Boat or Ferry,Bookstore,Breakfast Spot,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Business Service,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comedy Club,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Food,Food Court,Food Truck,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,Gay Bar,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Himalayan Restaurant,History Museum,Hostel,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Italian Restaurant,Japanese Restaurant,Jazz Club,Kids Store,Lake,Lebanese Restaurant,Liquor Store,Locksmith,Luggage Store,Malay Restaurant,Market,Massage Studio,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Music Venue,Night Market,Nightclub,Noodle House,Office,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Outlet Mall,Outlet Store,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Poke Place,Pool,Pub,Ramen Restaurant,Real Estate Office,Recreation Center,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Soup Place,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stationery Store,Supermarket,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Trail,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Shop,Women's Store,Yoga Studio
0,"North Hastings, Sunrise",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"North Hastings, Sunrise",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,"North Hastings, Sunrise",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"North Hastings, Sunrise",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"North Hastings, Sunrise",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [119]:
Vancouver_onehot.shape

(696, 151)

In [120]:
Vancouver_grouped=Vancouver_onehot.groupby('Neighborhood').mean().reset_index()
Vancouver_grouped

,Neighborhood,Accessories Store,American Restaurant,Arts & Crafts Store,Asian Restaurant,Bagel Shop,Bakery,Bank,Bar,Beach,Big Box Store,Bistro,Boat Rental,Boat or Ferry,Bookstore,Breakfast Spot,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Business Service,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comedy Club,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Food,Food Court,Food Truck,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,Gay Bar,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Himalayan Restaurant,History Museum,Hostel,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Italian Restaurant,Japanese Restaurant,Jazz Club,Kids Store,Lake,Lebanese Restaurant,Liquor Store,Locksmith,Luggage Store,Malay Restaurant,Market,Massage Studio,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Music Venue,Night Market,Nightclub,Noodle House,Office,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Outlet Mall,Outlet Store,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Poke Place,Pool,Pub,Ramen Restaurant,Real Estate Office,Recreation Center,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Soup Place,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stationery Store,Supermarket,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Trail,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Shop,Women's Store,Yoga Studio
0,"West Kensington, Cedar Cottage, North East Ri...",0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.041667,0.000000,0.041667,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.166667,0.000000,0.000000,0.000000
1,Bental Center,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

In [121]:
Vancouver_grouped.shape

(29, 151)

In [122]:
num_top_venues = 5

for hood in Vancouver_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Vancouver_grouped[Vancouver_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- West Kensington, Cedar Cottage, North East Riley Park, Little Mountain ----
                           venue  freq
0          Vietnamese Restaurant  0.17
1  Vegetarian / Vegan Restaurant  0.08
2              Indian Restaurant  0.04
3                 Breakfast Spot  0.04
4                  Grocery Store  0.04


----Bental Center ----
                venue  freq
0       Grocery Store  0.11
1  Dim Sum Restaurant  0.06
2      Hardware Store  0.06
3         Coffee Shop  0.06
4      Shop & Service  0.06


----Central Kitsilano, Greektown ----
                 venue  freq
0          Coffee Shop  0.13
1  Japanese Restaurant  0.07
2          Yoga Studio  0.03
3         Burger Joint  0.03
4       Farmers Market  0.03


----East Fairview, South Cambie ----
                 venue  freq
0          Coffee Shop  0.13
1          Bus Station  0.10
2                  Gym  0.07
3  Japanese Restaurant  0.07
4          Pizza Place  0.07


----East Mount Pleasant----
             venue  freq
0         

In [123]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [124]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Vancouver_grouped['Neighborhood']

for ind in np.arange(Vancouver_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Vancouver_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"West Kensington, Cedar Cottage, North East Ri...",Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Construction & Landscaping,Smoke Shop,South Indian Restaurant,Breakfast Spot,Bus Stop,Chinese Restaurant,Supermarket,Grocery Store
1,Bental Center,Grocery Store,Pizza Place,Big Box Store,Shop & Service,Clothing Store,Burger Joint,Liquor Store,Coffee Shop,Sporting Goods Shop,Sushi Restaurant
2,"Central Kitsilano, Greektown",Coffee Shop,Japanese Restaurant,Yoga Studio,Donut Shop,Malay Restaurant,Gym / Fitness Center,Greek Restaurant,Furniture / Home Store,Frozen Yogurt Shop,Farmers Market
3,"East Fairview, South Cambie",Coffee Shop,Bus Station,Gym,Japanese Restaurant,Pizza Place,Burger Joint,Sandwich Place,Skating Rink,Fast Food Restaurant,Cantonese Restaurant
4,East Mount Pleasant,Bus Stop,Pizza Place,Park,Bubble Tea Shop,Gift Shop,Café,Grocery Store,Fast Food Restaurant,Food Truck,Food Court


Run *k*-means to cluster the neighborhood into 5 clusters.

In [125]:
from sklearn.cluster import KMeans
def calculate_WSS (points, kmax):
    sse=[]
    for k in range (1, kmax+1):
        kmeans = KMeans(n_cluster = k).fit(points)
        centroids=kmeans.cluster_center_
        pred_clusters=kmeans.predict(points)
        curr_sse = 0 
        for i in range (len(points)):
            curr_center=centroids[pred_cluster[i]]
            curr_sse += (points[i, 0] - curr_center[0])**2 +(points[i, 1] - curr_center[1])
        sse.append(curr_sse)
    return sse

In [126]:
# set number of clusters
kclusters = 5

Vancouver_grouped_clustering = Vancouver_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Vancouver_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 2, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [127]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Vancouver_merged = Van_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Vancouver_merged = Vancouver_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Vancouver_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,V5K,Vancouver,"North Hastings, Sunrise",-123.142289,49.303832,2.0,Trail,Park,Lake,Event Space,Food Truck,Food Court,Food,Fish Market,Fast Food Restaurant,Farmers Market
1,V5L,Vancouver,"North grandview, Woodland",-123.142289,49.303832,2.0,Trail,Park,Lake,Event Space,Food Truck,Food Court,Food,Fish Market,Fast Food Restaurant,Farmers Market
2,V5M,Vancouver,"South Hastings, Sunrise, North Renfrew- Collin...",-123.034651,49.249830,0.0,Pizza Place,Malay Restaurant,Convenience Store,Bookstore,Supermarket,Sushi Restaurant,Farmers Market,Food Court,Food,Fish Market
3,V5N,Vancouver,"South Grandview, Woodland, Ne Kensington, Ceda...",-123.067986,49.249830,0.0,Vietnamese Restaurant,Burger Joint,Seafood Restaurant,Bank,Sandwich Place,Chinese Restaurant,Café,Grocery Store,Yoga Studio,Fast Food Restaurant
4,V5P,Vancouver,"SE Kensington, Cedar Cottage, Victoria drive, ...",-123.067984,49.216496,3.0,Middle Eastern Restaurant,Japanese Restaurant,Motorcycle Shop,Yoga Studio,Farmers Market,French Restaurant,Food Truck,Food Court,Food,Fish Market


In [134]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Vancouver_merged['Latitude'], Vancouver_merged['Longitude'], Vancouver_merged['Neighborhood'], Vancouver_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [129]:
Vancouver_merged.loc[Vancouver_merged['Cluster Labels'] == 0, Vancouver_merged.columns[[1] + list(range(5, Vancouver_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Vancouver,0.0,Pizza Place,Malay Restaurant,Convenience Store,Bookstore,Supermarket,Sushi Restaurant,Farmers Market,Food Court,Food,Fish Market
3,Vancouver,0.0,Vietnamese Restaurant,Burger Joint,Seafood Restaurant,Bank,Sandwich Place,Chinese Restaurant,Café,Grocery Store,Yoga Studio,Fast Food Restaurant
5,Vancouver,0.0,Trail,Theme Park Ride / Attraction,History Museum,Music Venue,Playground,Park,Bus Station,Pub,Outdoor Sculpture,Other Great Outdoors
6,Vancouver,0.0,Pharmacy,Grocery Store,Shopping Mall,Sandwich Place,Chinese Restaurant,Recreation Center,Video Store,Pizza Place,Deli / Bodega,Dim Sum Restaurant
7,Vancouver,0.0,Bus Stop,Pizza Place,Park,Bubble Tea Shop,Gift Shop,Café,Grocery Store,Fast Food Restaurant,Food Truck,Food Court
8,Vancouver,0.0,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Construction & Landscaping,Smoke Shop,South Indian Restaurant,Breakfast Spot,Bus Stop,Chinese Restaurant,Supermarket,Grocery Store
11,Vancouver,0.0,Coffee Shop,Japanese Restaurant,Chinese Restaurant,Pizza Place,Fast Food Restaurant,Gym,Sushi Restaurant,Café,Mexican Restaurant,Sporting Goods Shop
12,Vancouver,0.0,Coffee Shop,Bus Station,Gym,Japanese Restaurant,Pizza Place,Burger Joint,Sandwich Place,Skating Rink,Fast Food Restaurant,Cantonese Restaurant
13,Vancouver,0.0,Pub,Park,Locksmith,Yoga Studio,Falafel Restaurant,Food Court,Food,Fish Market,Fast Food Restaurant,Farmers Market
15,Vancouver,0.0,Café,Bakery,Mexican Restaurant,Japanese Restaurant,Gay Bar,Coffee Shop,Indian Restaurant,Sushi Restaurant,Park,Grocery Store


In [130]:
Vancouver_merged.loc[Vancouver_merged['Cluster Labels'] == 1, Vancouver_merged.columns[[1] + list(range(5, Vancouver_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Vancouver,1.0,Scenic Lookout,Yoga Studio,Food Truck,Food Court,Food,Fish Market,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


In [131]:
Vancouver_merged.loc[Vancouver_merged['Cluster Labels'] == 2, Vancouver_merged.columns[[1] + list(range(5, Vancouver_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Vancouver,2.0,Trail,Park,Lake,Event Space,Food Truck,Food Court,Food,Fish Market,Fast Food Restaurant,Farmers Market
1,Vancouver,2.0,Trail,Park,Lake,Event Space,Food Truck,Food Court,Food,Fish Market,Fast Food Restaurant,Farmers Market
16,Vancouver,2.0,Trail,Bus Stop,Garden,Park,Farmers Market,French Restaurant,Food Truck,Food Court,Food,Fish Market


In [132]:
Vancouver_merged.loc[Vancouver_merged['Cluster Labels'] == 3, Vancouver_merged.columns[[1] + list(range(5, Vancouver_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Vancouver,3.0,Middle Eastern Restaurant,Japanese Restaurant,Motorcycle Shop,Yoga Studio,Farmers Market,French Restaurant,Food Truck,Food Court,Food,Fish Market


In [133]:
Vancouver_merged.loc[Vancouver_merged['Cluster Labels'] == 4, Vancouver_merged.columns[[1] + list(range(5, Vancouver_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Vancouver,4.0,Business Service,Park,Electronics Store,Farmers Market,French Restaurant,Food Truck,Food Court,Food,Fish Market,Fast Food Restaurant
10,Vancouver,4.0,Business Service,Park,Electronics Store,Farmers Market,French Restaurant,Food Truck,Food Court,Food,Fish Market,Fast Food Restaurant
